In [ ]:
import os
from pathlib import Path
import pandas as pd
import pickle
import numpy as np

# # df = pd.read_csv('data/selected_triplets.csv').dropna()
# df = pd.read_csv('data/selected_triplets.csv').dropna()

# os.listdir('../models/11_03')
# X_encoded_dict = np.load('../models/11_03/x_encoded_dict.pkl', allow_pickle=True)
# X_encoded_dict = {os.path.split(k)[-1][:4]: v for k, v in X_encoded_dict.items()}

In [ ]:
from collections import Counter, defaultdict


df_file = Path('data/selected_triplets.csv')
fixed_df_file = Path('selected_triplets_fixed.csv')
lengths = []
lengths_dict = defaultdict(list)
lines = []
with open(df_file, 'r') as file:
    for i, line in enumerate(file.readlines()):
        if i == 0 or len(line.split(',')) == 9:
            lines.append(line)
        lengths.append(len(line.split(',')))
        lengths_dict[len(line.split(','))].append((i, line))

with open(fixed_df_file, 'w') as file:
    file.writelines(lines)

In [29]:
df = pd.read_csv(fixed_df_file)
df['file_id'] = df.video_file.map(lambda p: int(os.path.split(p)[-1][:4]))
df = df[df.file_id == 37]
df.selected.value_counts()

0    3596
1    1560
2    1479
Name: selected, dtype: int64

In [ ]:
df['file_id'] = df.video_file.map(lambda p: os.path.split(p)[-1][:4])
verified_df = df[df.selected == df.selected_verification]

In [ ]:
def get_segment_enc(encodings, seg_string):
    start, end = seg_string[1:-1].split(',')
    start, end = int(start), int(end)
    
    start_idx, end_idx = start // 4 , end // 4
    assert end_idx > start_idx, print(start_idx, end_idx)
    idx = (start_idx + end_idx) // 2
    if idx >= len(encodings):
        return None
    return encodings[start_idx]
    return encodings[start_idx:end_idx].mean(axis=0)

In [ ]:
df.anchor.map(lambda o: get_segment_enc(X_encoded_dict['0015'], o))
for sample in ['anchor', 'sample1', 'sample2']:
    df[f'{sample}_enc'] = df[sample].map(lambda o: get_segment_enc(X_encoded_dict['0015'], o))
    

In [ ]:
df['d1'] = (df['anchor_enc'] - df['sample1_enc']).map(np.linalg.norm)
df['d2'] = (df['anchor_enc'] - df['sample2_enc']).map(np.linalg.norm)
df['d3'] = (df['sample1_enc'] - df['sample2_enc']).map(np.linalg.norm)
df['pred_selected'] = (df['d1'] <= df['d2']).map(lambda b: '1' if b else '2')
df['pred_selected1'] = (df['d1'] + 20 <= df['d2'])#.map(lambda b: '1' if b else '2')
df['pred_selected2'] = (df['d2']  + 20<= df['d1'])#.map(lambda b: '1' if b else '2')
df['pred_correct'] = (df.selected == df.pred_selected)
df.describe()

In [ ]:
df[df['selected'] != '0']['pred_correct'].mean()

In [ ]:
verified_df = df[df.selected == df.selected_verification]
verified_df[verified_df['selected'] != '0']['pred_correct'].mean()
verified_df

In [ ]:
df[df['selected'] != '0'][['selected', 'pred_selected']].iloc[:10]

In [ ]:
df.iloc[2], X_encoded_dict.keys()